In [1]:
from IPython.display import FileLink, FileLinks

data_path = 'src/real data/2017 CAM data from iPads'
FileLinks(data_path)

src/real data/2017 CAM data from iPads/
  2017 CAM data from iPads - Devvyn.xlsx
  input_ Combined Erl_ Tyler Query.xlsx
  2017 CAM data from iPads.xlsx

In [2]:
filename = "2017 CAM data from iPads.xlsx"

In [3]:
import pandas
from pathlib import PurePath

data_file = pandas.ExcelFile(PurePath(data_path, filename))
sheets = {sheet_name:data_file.parse(sheet_name) for sheet_name in data_file.sheet_names}

In [4]:
display(list(sheets))

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

In [5]:
from collections import defaultdict
import json



In [12]:
def split_column_to_dict(sheet, column, coldic=None, separator='__'):
    if coldic is None:
        nested_dict = lambda: defaultdict(nested_dict)
        coldic = nested_dict()
    colpointer_previous = colpointer = coldic
    for segment in str(column).split(separator):
        display(segment, colpointer_previous)
        colpointer_previous = colpointer
        colpointer = colpointer[segment]
    endpoint = colpointer_previous[segment] = sheet[column]  # fixme! conflicts with pandas if attribute name collides
    return coldic, endpoint

column_dictionary = dict()
for sheet_name, sheet in list(sheets.items())[2:3]:
    display(sheet_name)
    coldic = None
    for column in sheet.columns[-8:-6]:
        display(str(column))
        coldic, endpoint = split_column_to_dict(sheet, column, coldic)
        #display(endpoint.dropna()[:10])
    column_dictionary[sheet_name] = coldic


'2017 CAM iPad data Tyler'

'fields__oSets__oPoints__observations__|'

'fields'

defaultdict(<function __main__.split_column_to_dict.<locals>.<lambda>>, {})

'oSets'

...

'|'

defaultdict(<function __main__.split_column_to_dict.<locals>.<lambda>>,
            {'observations': defaultdict(<function __main__.split_column_to_dict.<locals>.<lambda>>,
                         {'|': 0       NaN
                          1       NaN
                          2       NaN
                          3       NaN
                          4       NaN
                          5        e1
                          6        e2
                          7        e3
                          8        e4
                          9        e5
                          10       e6
                          11       e7
                          12       e8
                          13       e9
                          14      NaN
                          15      NaN
                          16      NaN
                          17      NaN
                          18      NaN
                          19       e1
                          20       e2
                        

'number'

defaultdict(<function __main__.split_column_to_dict.<locals>.<lambda>>,
            {'|': 0       NaN
             1       NaN
             2       NaN
             3       NaN
             4       NaN
             5        e1
             6        e2
             7        e3
             8        e4
             9        e5
             10       e6
             11       e7
             12       e8
             13       e9
             14      NaN
             15      NaN
             16      NaN
             17      NaN
             18      NaN
             19       e1
             20       e2
             21       e3
             22       e4
             23       e5
             24       e6
             25       e7
             26       e8
             27       e9
             28      NaN
             29      NaN
                    ... 
             3750     e8
             3751     e9
             3752    NaN
             3753    NaN
             3754    NaN
             3755    Na

KeyError: 'number'

In [ ]:
sheet = sheets['2017 CAM iPad data Tyler']
coldic, data = split_column_to_dict(sheet, sheet.columns[20])
coldic, data = split_column_to_dict(sheet, sheet.columns[21], coldic)


In [ ]:
coldic['fields']['oSets']['oPoints']['observations'].keys()

In [ ]:
coldic, data = split_column_to_dict(sheet, sheet.columns[22], coldic)